# Lab5: Machine Translation

Machine translation is an application of NLP to computationally translate text from one language to the other. It is one of the most popular fields of research in the NLP community. Very recently (2014) deep learning methods changed the face of machine translation and all of NLP. The application of deep learning to machine translation was referred to as Neural Machine Translation (NMT).

In this mini lab, we will see a sample code that performs neural machine translation, and we will see the effects of various hyperparameters on the performance of the system.

For simplicity, we will take as our dataset, a set of short English sentences mapped to French sentences, and instead of performing the translation one word at a time, it will be performing it one character at a time.

Total points: 20 points + 10 bonus points

**Submission Instructions**: Just upload this notebook, with all your answers in the respective cells, to Compass.

The following block of code contains the full code of the functional NMT system. Run it to get a sense of how the loss and accuracy of the training data and the validation data are changing with every epoch. No data is necessary to download; all is included in the lab's directory.

#### Packages to install:
- pip install keras
- pip install tensorflow

In [2]:
pip install keras

     |████████████████████████████████| 378kB 5.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(8000,8010):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 16s 2ms/step - loss: 1.2332 - accuracy: 0.7241 - val_loss: 1.1049 - val_accuracy: 0.6965
Epoch 2/10
8000/8000 [==============================] - 17s 2ms/step - loss: 0.9115 - accuracy: 0.7478 - val_loss: 0.9643 - val_accuracy: 0.7365
Epoch 3/10
8000/8000 [==============================] - 16s 2ms/step - loss: 0.7925 - accuracy: 0.7855 - val_loss: 0.8345 - val_accuracy: 0.7712
Epoch 4/10
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6812 - accuracy: 0.8088 - val_loss: 0.7451 - val_accuracy: 0.7845
Epoch 5/10
8000/8000 [==============================] - 15s 2ms/step - loss: 0.6176 - accuracy: 0.8226 - val_loss: 0.6881 - val_accuracy: 0.7983
Epoch 6/10
8000/8000 [==============================] - 16s 2m

When training an NMT system (or any deep learning system), one of the decisions to take is the batch size: number of examples processed (in parallel) before updating parameters.

**Deliverable 1** (5 points): Change the batch size from 64 to 128, and then 256. How did it affect the speed of training (time per epoch)? Is it slower or faster? Why? What was its effect on the increase of accuracy from one epoch to the other? What is the concluded tradeoff between a smaller batch size and a larger one?

Another decision to make about the architecture of the NMT is the latent dimension size of the LSTM block. The LSTM block encodes semantics of a sentence to a vector of size equal to the latent dimension. This controls the modeling capacity of our system.

**Deliverable 2** (5 points): Change the latent dimension from a 100 to a 10, and a 1000. How did it affect speed of training? Why? How did affect the accuracy of the system?

**Deliverable 3** (5 points): One of the issues with the code above is that at demo time (for loop at the bottom of the code), the test is performed on training instances instead of validation instances. Adjust the code to perform the demo on the first 10 instances of the validation instances, and copy the cell's output below:

**Deliverable 4** (5 points): In the following piece of code `decoder_dense = Dense(num_decoder_tokens, activation='softmax')` a dense layer is instantiated to map the output of the LSTM block to a prediction of the next character. Why is the output size of this dense layer set to `num_decoder_tokens`, which is the number of possible French output characters?

**Bonus Question** (10 points): During training, the model, besides being fed the target data (French translation) as in the normal supervised procedure, it is also fed the target data shifted by one position to the left. What is the purpose of this practice?  

In [3]:
!tar chvfz notebook.tar.gz *

a Lab5.ipynb
a fra-eng
a fra-eng/_about.txt
a fra-eng/fra.txt
a notebook.tar.gz: Can't add archive to itself
a s2s.h5
